In [ ]:
#| hide
from fastcaddy.core import *

# fastcaddy

> A simple python wrapper for using the caddy API

## Usage

### Installation

Install from [pypi][pypi]


```sh
$ pip install fastcaddy
```

[pypi]: https://pypi.org/project/fastcaddy/

## Installing Caddy

In [ ]:
from fastcore.utils import *

This project is to help you use the caddy API, rather than a Caddyfile, to use caddy. To use the API, you need to install a plugin for your domain management service. We use Cloudflare, so we'll document that here. For other domain services, see the Caddy docs for other plugins.

### Cloudflare setup

You'll need a token from Cloudflare with access to modify the necessary settings. Here's the steps to create a token with the minimal privileges. You'll need to install the cloudflare pip package, then import:

In [ ]:
from cloudflare import Cloudflare

Then you'll need create a Cloudflare API token for your user, which we'll then use to create the less privileged token.

In [ ]:
cf_token = os.environ['CLOUDFLARE_API_TOKEN']

We can now check that works OK:

In [ ]:
cf = Cloudflare(api_token=cf_token)
zones = cf.zones.list()
len(zones.result)

8

Replace this with your domain name:

In [ ]:
domain = 'answer.ai'
zones = cf.zones.list(name=domain)
assert len(zones.result)==1

In [ ]:
zone_id = zones.result[0].id

Here's the methods available for modifying DNS records:

- `client.dns.records.create(*, zone_id, **params) -> Optional`
- `client.dns.records.update(dns_record_id, *, zone_id, **params) -> Optional`
- `client.dns.records.list(*, zone_id, **params) -> SyncV4PagePaginationArray[Record]`
- `client.dns.records.delete(dns_record_id, *, zone_id) -> Optional`
- `client.dns.records.edit(dns_record_id, *, zone_id, **params) -> Optional`
- `client.dns.records.export(*, zone_id) -> str`
- `client.dns.records.get(dns_record_id, *, zone_id) -> Optional`
- `client.dns.records.import\_(*, zone_id, **params) -> Optional`
- `client.dns.records.scan(*, zone_id, **params) -> Optional`

…and here's the methods for tokens:

```python
from cloudflare.types.user import (CIDRList, Policy, Token, TokenCreateResponse, TokenUpdateResponse, TokenListResponse,
                                   TokenDeleteResponse, TokenGetResponse, TokenVerifyResponse)
```

- `client.user.tokens.create(**params) -> Optional`
- `client.user.tokens.update(token_id, **params) -> object`
- `client.user.tokens.list(**params) -> SyncV4PagePaginationArray[object]`
- `client.user.tokens.delete(token_id) -> Optional`
- `client.user.tokens.get(token_id) -> object`
- `client.user.tokens.verify() -> Optional`

```python
from cloudflare.types.user.tokens import PermissionGroupListResponse
```

- client.user.tokens.permission_groups.list() -> SyncSinglePage[object]

```python
from cloudflare.types.user.tokens import Value
```

- client.user.tokens.value.update(token_id, **params) -> str

We need these two permissions in our token:

In [ ]:
permission_groups = cf.user.tokens.permission_groups.list()

dns_write = next(group for group in permission_groups if group['name'] == 'DNS Write')
zone_read = next(group for group in permission_groups if group['name'] == 'Zone Read')

Now we can create it:

```python
new_token = cf.user.tokens.create(
    name='caddy_dns',
    policies=[{
        "effect": "allow",
        "resources": { f"com.cloudflare.api.account.zone.{zone_id}": "*" },
        "permission_groups": [
            {"id": zone_read['id'], "name": "Zone Read"},
            {"id": dns_write['id'], "name": "DNS Write"}
        ]
    }]
)

print(new_token.value)
```

Make a copy of this value, which we'll need for setting up caddy.

### Installing caddy

To install caddy, we'll use a tool called `xcaddy`. This is written in go. So first install go:

- Mac: `brew install go`
- Linux: `sudo apt install golang`

Now we can install xcaddy:

```sh
go install github.com/caddyserver/xcaddy/cmd/xcaddy@latest
```

Then we use that to compile caddy with our desired domain plugin (cloudflare, in this case):

```sh
cd ~/go/bin
./xcaddy build --with github.com/caddy-dns/cloudflare
```

This gives us a `~/go/bin/caddy` binary we can run:

```sh
./caddy run
```

### Auto-run caddy on start

If you're using a server or running caddy a lot, you'll want it to run on start. This isn't needed otherwise -- you can just `~/go/bin/caddy run` to run it manually (you may want to add `~/go/bin` to your `PATH` env var).

## How to use